In [1]:
import utils
import params
import os
from tqdm import tqdm
import numpy as np
import xlsxwriter
from collections import defaultdict

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.preprocessing import image

C:\Users\bldr1\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### CHOOSE 1000 TEST QUERIES

In [2]:
captions = utils.readCaptions(params.VAL_CAPTIONS_PATH)

In [3]:
captions[0]

{'caption': 'A bicycle replica with a clock as the front wheel.',
 'id': 37,
 'image_id': 203564}

In [4]:
"""N = 1011
captions_idxs = np.arange(captions.__len__())
np.random.shuffle(captions_idxs)"""

'N = 1011\ncaptions_idxs = np.arange(captions.__len__())\nnp.random.shuffle(captions_idxs)'

In [5]:
"""captions_dict = {}
for caption_idx in tqdm(captions_idxs[:N]):
    caption = captions[caption_idx]['caption']
    caption_id = captions[caption_idx]['image_id']
    caption_tfidf = np.load( params.TFIDF_DESCRIPTORS_VAL_PATH+str(caption_id)+'.npy').tolist()[0]
    caption_rtfidf = np.load( params.REDUCED_TFIDF_DESCRIPTORS_VAL_PATH+str(caption_id)+'.npy').tolist()[0]
    caption_w2v = np.load(  params.W2V_DESCRIPTORS_VAL_PATH+str(caption_id)+'.npy' )[0]
    captions_dict[str(caption_id)] = {}
    captions_dict[str(caption_id)]['caption'] = caption
    captions_dict[str(caption_id)]['tfidf'] = caption_tfidf
    captions_dict[str(caption_id)]['r-tfidf'] = caption_rtfidf
    captions_dict[str(caption_id)]['w2v'] = caption_w2v"""

"captions_dict = {}\nfor caption_idx in tqdm(captions_idxs[:N]):\n    caption = captions[caption_idx]['caption']\n    caption_id = captions[caption_idx]['image_id']\n    caption_tfidf = np.load( params.TFIDF_DESCRIPTORS_VAL_PATH+str(caption_id)+'.npy').tolist()[0]\n    caption_rtfidf = np.load( params.REDUCED_TFIDF_DESCRIPTORS_VAL_PATH+str(caption_id)+'.npy').tolist()[0]\n    caption_w2v = np.load(  params.W2V_DESCRIPTORS_VAL_PATH+str(caption_id)+'.npy' )[0]\n    captions_dict[str(caption_id)] = {}\n    captions_dict[str(caption_id)]['caption'] = caption\n    captions_dict[str(caption_id)]['tfidf'] = caption_tfidf\n    captions_dict[str(caption_id)]['r-tfidf'] = caption_rtfidf\n    captions_dict[str(caption_id)]['w2v'] = caption_w2v"

In [6]:
"""captions_dict.keys().__len__()"""

'captions_dict.keys().__len__()'

In [7]:
"""utils.savePickle(object=captions_dict, PATH='TMP/test_captions')"""

"utils.savePickle(object=captions_dict, PATH='TMP/test_captions')"

In [8]:
"""test_captions = utils.importPickle('TMP/test_captions')"""

In [ ]:
OUTPUT_SHAPE = 128
TFIDF_CNN_WEIGHTS = 'TMP/model_tfidf_cnn_pca_dropout_02.h5'
RTFIDF_CNN_WEIGHTS = 'TMP/model_reduced_tfidf_cnn_pca_dropout_02.h5'
W2V_CNN_WEIGHTS = 'TMP/model_w2v_cnn_pca_dropout_02.h5'

In [ ]:
INPUT_SHAPE = 369537
tfidf_model = Sequential()
tfidf_model.add(Dense(input_dim=INPUT_SHAPE, units = 150, kernel_initializer='normal'))
tfidf_model.add(Activation('relu'))
tfidf_model.add(Dropout(0.2))
tfidf_model.add(Dense(units = 130, kernel_initializer='normal'))
tfidf_model.add(Activation('relu'))
tfidf_model.add(Dropout(0.2))
tfidf_model.add(Dense(units = OUTPUT_SHAPE, kernel_initializer='normal'))
tfidf_model.compile(loss='mean_squared_error',optimizer='adam')

tfidf_model.load_weights(TFIDF_CNN_WEIGHTS)

In [ ]:
INPUT_SHAPE = 91155
rtfidf_model = Sequential()
rtfidf_model.add(Dense(input_dim=INPUT_SHAPE, units = 150, kernel_initializer='normal'))
rtfidf_model.add(Activation('relu'))
rtfidf_model.add(Dropout(0.2))
rtfidf_model.add(Dense(units = 130, kernel_initializer='normal'))
rtfidf_model.add(Activation('relu'))
rtfidf_model.add(Dropout(0.2))
rtfidf_model.add(Dense(units = OUTPUT_SHAPE, kernel_initializer='normal'))
rtfidf_model.compile(loss='mean_squared_error',optimizer='adam')

rtfidf_model.load_weights(RTFIDF_CNN_WEIGHTS)

In [33]:
INPUT_SHAPE = 300
w2v_model = Sequential()
w2v_model.add(Dense(input_dim=INPUT_SHAPE, units = 150, kernel_initializer='normal'))
w2v_model.add(Activation('relu'))
w2v_model.add(Dropout(0.2))
w2v_model.add(Dense(units = 130, kernel_initializer='normal'))
w2v_model.add(Activation('relu'))
w2v_model.add(Dropout(0.2))
w2v_model.add(Dense(units = OUTPUT_SHAPE, kernel_initializer='normal'))
w2v_model.compile(loss='mean_squared_error',optimizer='adam')

w2v_model.load_weights(W2V_CNN_WEIGHTS)

In [34]:
tree = utils.importPickle(params.CNN_VAL_INDEX)
order = utils.importPickle(params.CNN_VAL_ORDER)

In [35]:
for key in tqdm(captions_dict.keys()):
    tfidf_pred = tfidf_model.predict(captions_dict[key]['tfidf'])
    rtfidf_pred = rtfidf_model.predict(captions_dict[key]['r-tfidf'])
    w2v_pred = w2v_model.predict(captions_dict[key]['w2v'].reshape(-1,300))
      
    tfidf_dist, tfidf_idxs = tree.query(tfidf_pred/np.linalg.norm(tfidf_pred), k=10)
    rtfidf_dist, rtfidf_idxs = tree.query(rtfidf_pred/np.linalg.norm(rtfidf_pred), k=10)
    w2v_dist, w2v_idxs = tree.query(w2v_pred/np.linalg.norm(w2v_pred), k=10)
    
    tfidf_images = [order[i] for i in tfidf_idxs[0]] 
    rtfidf_images = [order[i] for i in rtfidf_idxs[0]]
    w2v_images = [order[i] for i in w2v_idxs[0]]
    
    tfidf_paths = [params.VAL_IMAGES_PATH + os.path.basename(image).split('.')[0][4:]+'.jpg' for image in tfidf_images]
    rtfidf_paths = [params.VAL_IMAGES_PATH + os.path.basename(image).split('.')[0][4:]+'.jpg' for image in rtfidf_images]
    w2v_paths = [params.VAL_IMAGES_PATH + os.path.basename(image).split('.')[0][4:]+'.jpg' for image in w2v_images]
    
    captions_dict[key]['tfidf_paths'] = tfidf_paths
    captions_dict[key]['rtfidf_paths'] = rtfidf_paths
    captions_dict[key]['w2v_paths'] = w2v_paths

100%|██████████| 1000/1000 [01:52<00:00,  8.88it/s]


In [36]:
utils.savePickle(captions_dict,'TMP/cnn_captions')

In [37]:
alphabet = ['C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 
            'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [40]:
for counter in range(10):
    tfidf_workbook = xlsxwriter.Workbook('EXCELS/TFIDF_CNN/tfidf_cnn_{}.xlsx'.format(counter))
    tfidf_worksheet = tfidf_workbook.add_worksheet()
    tfidf_worksheet.set_column('A:A', 10)
    tfidf_worksheet.set_column('B:B', 60)
    tfidf_worksheet.write('A1','POSITION')

    j = 4
    for key in tqdm(list(captions_dict.keys())[counter*100:counter*100+100]):
        caption = captions_dict[key]['caption']
        tfidf_paths = captions_dict[key]['tfidf_paths']
        #rtfidf_paths = captions_dict[key]['rtfidf_paths']
        #w2v_paths = captions_dict[key]['w2v_paths']



        tfidf_worksheet.write('B{}'.format(j), caption)
        for i, path in enumerate(tfidf_paths):
            img = image.load_img(path, target_size=(224, 224))
            name = 'CROP/'+os.path.basename(path)
            image.save_img(name, img)
            tfidf_worksheet.insert_image('{}{}'.format(alphabet[i*2], j-2), 
                                         name, 
                                         {'x_scale': 0.5, 'y_scale': 0.5})

        j = j+6
    tfidf_workbook.close()

100%|██████████| 100/100 [00:27<00:00,  3.63it/s]


In [41]:
for counter in range(10):
    rtfidf_workbook = xlsxwriter.Workbook('EXCELS/R-TFIDF_CNN/rtfidf_cnn_{}.xlsx'.format(counter))
    rtfidf_worksheet = rtfidf_workbook.add_worksheet()
    rtfidf_worksheet.set_column('A:A', 10)
    rtfidf_worksheet.set_column('B:B', 60)
    rtfidf_worksheet.write('A1','POSITION')

    j = 4
    for key in tqdm(list(captions_dict.keys())[counter*100:counter*100+100]):
        caption = captions_dict[key]['caption']
        rtfidf_paths = captions_dict[key]['rtfidf_paths']
        #rtfidf_paths = captions_dict[key]['rtfidf_paths']
        #w2v_paths = captions_dict[key]['w2v_paths']



        rtfidf_worksheet.write('B{}'.format(j), caption)
        for i, path in enumerate(rtfidf_paths):
            img = image.load_img(path, target_size=(224, 224))
            name = 'CROP/'+os.path.basename(path)
            image.save_img(name, img)
            rtfidf_worksheet.insert_image('{}{}'.format(alphabet[i*2], j-2), 
                                         name, 
                                         {'x_scale': 0.5, 'y_scale': 0.5})

        j = j+6
    rtfidf_workbook.close()

100%|██████████| 100/100 [00:28<00:00,  3.55it/s]


In [42]:
for counter in range(10):
    w2v_workbook = xlsxwriter.Workbook('EXCELS/W2V_CNN/w2v_cnn_{}.xlsx'.format(counter))
    w2v_worksheet = w2v_workbook.add_worksheet()
    w2v_worksheet.set_column('A:A', 10)
    w2v_worksheet.set_column('B:B', 60)
    w2v_worksheet.write('A1','POSITION')

    j = 4
    for key in tqdm(list(captions_dict.keys())[counter*100:counter*100+100]):
        caption = captions_dict[key]['caption']
        w2v_paths = captions_dict[key]['w2v_paths']
        #rtfidf_paths = captions_dict[key]['rtfidf_paths']
        #w2v_paths = captions_dict[key]['w2v_paths']



        w2v_worksheet.write('B{}'.format(j), caption)
        for i, path in enumerate(w2v_paths):
            img = image.load_img(path, target_size=(224, 224))
            name = 'CROP/'+os.path.basename(path)
            image.save_img(name, img)
            w2v_worksheet.insert_image('{}{}'.format(alphabet[i*2], j-2), 
                                         name, 
                                         {'x_scale': 0.5, 'y_scale': 0.5})

        j = j+6
    w2v_workbook.close()

100%|██████████| 100/100 [00:25<00:00,  3.85it/s]
